#Escolha uma base de classificação e compare os classificadores Logistic Regression e KNN do scikit-learn.

##Use pelo menos duas formas de avaliação e as repita pelo menos 10 vezes.

##Calcule a média das repetições de cada avaliação.

* **Dataset:** https://archive.ics.uci.edu/ml/datasets/Car+Evaluation
* **Objetivo:** Classificação para identificar o conceito do veículo: unacc, acc, good, vgood

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import average_precision_score, matthews_corrcoef, accuracy_score, mean_squared_error, make_scorer
import numpy as np

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data

--2020-11-27 10:47:33--  https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51867 (51K) [application/x-httpd-php]
Saving to: ‘car.data.1’

car.data.1          100%[===================>]  50.65K   195KB/s    in 0.3s    

2020-11-27 10:47:34 (195 KB/s) - ‘car.data.1’ saved [51867/51867]



In [3]:
df = pd.read_csv("car.data",header=None)
df.columns = ["buying","maint","doors","persons","lug_boot","safety","concept"]
df

,buying,maint,doors,persons,lug_boot,safety,concept
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [4]:
X = df[["buying","maint","doors","persons","lug_boot","safety"]]
y = df[["concept"]]

In [5]:
le = LabelEncoder()
X = X.apply(le.fit_transform)
y = y.apply(le.fit_transform)

X.shape, y.shape

((1728, 6), (1728, 1))

In [6]:
resultados = []

In [7]:
pipeline = Pipeline([
    ("padronização", StandardScaler()),
    ("knn", KNeighborsClassifier())
])

parametros = {'knn__n_neighbors': [3,5,7,9,11]}

#modelo = GridSearchCV(pipeline, parametros, scoring='average_precision')
#Tentei fazer com o scoring acima porém apresentava erro
#FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: ValueError: multiclass format is not supported
#Não consegui entender ainda o motivo desse erro, se puder me orientar.
#Pesquisei várias alternativas porém sem sucesso de solução. Optei por manter, indevidamente, o score que você utilizou no exemplo de regressão.


modelo = GridSearchCV(pipeline, parametros, scoring='neg_root_mean_squared_error')

#scores = cross_validate(modelo, X, y, cv=10, scoring=make_scorer(average_precision_score))
scores = cross_validate(modelo, X, y, cv=10, scoring=make_scorer(mean_squared_error, squared=False))

print(scores['test_score'])
compad = np.mean(scores['test_score'])
print(f"Com padronização: {compad}")

resultados.append(("knn", "média mse", compad))


/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[1.05622296 0.96469469 0.56894589 0.92492774 0.48682052 0.80461099
 0.85679872 0.7091475  0.5        0.64699664]
Com padronização: 0.7519165640569496


/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

In [8]:
pipeline = Pipeline([
    ("padronização", StandardScaler()),
    ("knn", KNeighborsClassifier())
])

parametros = {'knn__n_neighbors': [3,5,7,9,11]}

modelo = GridSearchCV(pipeline, parametros, scoring='accuracy')

scores = cross_validate(modelo, X, y, cv=10, scoring=make_scorer(accuracy_score))
print(scores['test_score'])
compad = np.mean(scores['test_score'])
print(f"Com padronização: {compad}")

resultados.append(("knn", "média acuracy score", compad))


/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[0.70520231 0.74566474 0.89595376 0.78034682 0.94219653 0.75144509
 0.84971098 0.92485549 0.94767442 0.89534884]
Com padronização: 0.8438398978357308


/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

In [9]:
pipeline = Pipeline([
    ("padronização", StandardScaler()),
    ("lr", LogisticRegression())
])

parametros = {'lr__max_iter': [50,100,500]}

#modelo = GridSearchCV(pipeline, parametros, scoring='average_precision')

#Tentei fazer com o scoring acima porém apresentava erro
#FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: ValueError: multiclass format is not supported
#Não consegui entender ainda o motivo desse erro, se puder me orientar.
#Pesquisei várias alternativas porém sem sucesso de solução. Optei por manter, indevidamente, o score que você utilizou no exemplo de regressão.


modelo = GridSearchCV(pipeline, parametros, scoring='neg_root_mean_squared_error')

#scores = cross_validate(modelo, X, y, cv=10, scoring=make_scorer(average_precision_score))
scores = cross_validate(modelo, X, y, cv=10, scoring=make_scorer(mean_squared_error, squared=False))

print(scores['test_score'])
compad = np.mean(scores['test_score'])
print(f"Com padronização: {compad}")

resultados.append(("LR", "média mse", compad))


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

[1.04522034 1.21645747 1.23765581 1.18516846 1.20691648 1.20930879
 1.09649995 1.13280051 1.11543088 1.08638509]
Com padronização: 1.1531843777778554


In [10]:
pipeline = Pipeline([
    ("padronização", StandardScaler()),
    ("lr", LogisticRegression())
])

parametros = {'lr__max_iter': [50,100,500]}

modelo = GridSearchCV(pipeline, parametros, scoring='accuracy')

scores = cross_validate(modelo, X, y, cv=10, scoring=make_scorer(accuracy_score))
print(scores['test_score'])
compad = np.mean(scores['test_score'])
print(f"Com padronização: {compad}")

resultados.append(("LR", "média acuracy score", compad))


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

[0.69942197 0.58959538 0.66473988 0.6300578  0.64739884 0.6300578
 0.69942197 0.66473988 0.6744186  0.68023256]
Com padronização: 0.6580084688802258


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
resultados

[('knn', 'média mse', 0.7519165640569496),
 ('knn', 'média acuracy score', 0.8438398978357308),
 ('LR', 'média mse', 1.1531843777778554),
 ('LR', 'média acuracy score', 0.6580084688802258)]